---
# Imports

In [1]:
# imports.py
from imports import *
from sklearn.datasets import make_blobs

# plotting magic
%matplotlib inline

# custom mods
import tidy
import model_h
import wrangle

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


---
# Wrangle

In [2]:
df=wrangle.wrangle_df()

# df.info()
# # model_h.print_btcusd(df)
# df.head()

cached csv


In [3]:
df['atr'] = talib.ATR(df.high, df.low, df.close, 14)

In [4]:
df=df.loc['2021':]

---
# Train / Validate Split*

In [5]:
train, val=tidy.split_i(df)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 446 entries, 2021-01-01 to 2022-03-22
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   open                          446 non-null    float64
 1   high                          446 non-null    float64
 2   low                           446 non-null    float64
 3   close                         446 non-null    float64
 4   volume                        446 non-null    float64
 5   fwd_log_ret                   446 non-null    float64
 6   fwd_ret                       446 non-null    float64
 7   fwd_pct_chg                   446 non-null    float64
 8   fwd_close_positive            446 non-null    float64
 9   macd                          446 non-null    float64
 10  signal                        446 non-null    float64
 11  histo                         446 non-null    float64
 12  cross                         446 non-null   

---
# Baseline


In [7]:
(train.close[-1]-train.close[0])/len(train)

# UPDATE: DAILY simple return
# this is simple return if you bought 1 btc in 2014-09-17 and sold in 2022-03-23


29.113579035874448

In [8]:
(train.close.sum()-train.open.sum())/len(train)

# UPDATE: DAILY simple return
# this is simple return if you bought every morning and sold every night

19.852074840803517

---
# Model Results

In [9]:
df1=pd.read_csv('classi_train.csv').drop('Unnamed: 0',axis=1).set_index(train.index)
df2=pd.read_csv('classi_val.csv').drop('Unnamed: 0',axis=1).set_index(val.index)
df3=pd.read_csv('reg_val.csv').drop('Unnamed: 0',axis=1).set_index(val.index)
df4=pd.read_csv('reg_train.csv').drop('Unnamed: 0',axis=1).set_index(train.index)

In [10]:
df4.describe()

,fwd_ret,reg
count,446.000000,446.000000
mean,0.002604,0.002604
std,0.913825,0.163434
min,-3.777211,-0.391936
25%,-0.499245,-0.121946
50%,0.000000,-0.006409
75%,0.500755,0.109792
max,3.622124,0.531816


In [11]:
train=pd.concat([train,df1,df4.reg],axis=1)
val=pd.concat([val,df2,df3],axis=1)

In [12]:
profit=[]

for row in train.index:
	if (train.reg.loc[row]>=-10.007)&(train['class'].loc[row]==1):
		profit.append(train.close.loc[row+pd.DateOffset(days=1)]-train.open.loc[row+pd.DateOffset(days=1)])
		# print(train.close.loc[row+pd.DateOffset(days=1)]-train.open.loc[row+pd.DateOffset(days=1)])
	if (train.reg.loc[row]<-10.007)&(train['class'].loc[row]==0):
		profit.append(train.open.loc[row+pd.DateOffset(days=1)]-train.close.loc[row+pd.DateOffset(days=1)])
		# train.loc[row,'profit']=(df.open-df.close)
	if (row+pd.DateOffset(days=1))>=train.index[-1]:
		break
	else:
		profit.append(0)
		# train.loc[row,'profit']=(0)



In [13]:
sum(profit)/len(train)

69.67448499327357

---
# Validate

## *Baseline*

In [14]:
(val.close[-1]-val.close[0])/len(val)

# UPDATE: DAILY simple return
# this is simple return if you bought 1 btc in 2014-09-17 and sold in 2022-03-23


-106.44458006249988

In [15]:
(val.close.sum()-val.open.sum())/len(val)

# UPDATE: DAILY simple return
# this is simple return if you bought every morning and sold every night

-88.19482428125048

## *Val Profit*

In [17]:
profit_val=[]

for row in val.index:
	if (val.reg.loc[row]>=-10.007)&(val['class'].loc[row]==1):
		profit_val.append(val.close.loc[row+pd.DateOffset(days=1)]-val.open.loc[row+pd.DateOffset(days=1)])
		# print(val.close.loc[row+pd.DateOffset(days=1)]-val.open.loc[row+pd.DateOffset(days=1)])
	if (val.reg.loc[row]<-10.007)&(val['class'].loc[row]==0):
		profit_val.append(val.open.loc[row+pd.DateOffset(days=1)]-val.close.loc[row+pd.DateOffset(days=1)])
		# val.loc[row,'profit_val']=(df.open-df.close)
	if (row+pd.DateOffset(days=1))>=val.index[-1]:
		break
	else:
		profit_val.append(0)
		# val.loc[row,'profit_val']=(0)

In [18]:
sum(profit_val)/len(val)

-113.43713384374973